이 자료는 위키독스 딥 러닝을 이용한 자연어 처리 입문의 워드 임베딩의 평균 튜토리얼 자료입니다.  

링크 : https://wikidocs.net/103496

이 자료는 2021년 10월 13일에 마지막으로 테스트되었습니다.

# 09-12 문서 임베딩 : 워드 임베딩의 평균(Average Word Embedding)
---

앞서 특정 문장 내의 단어들의 임베딩 벡터들의 평균이 그 문장의 벡터가 될 수 있음을 설명했습니다. 이번에는 임베딩이 잘 된 상황에서는 단어 벡터들의 평균만으로 텍스트 분류를 수행할 수 있음을 보이고, 워드 임베딩의 중요성을 이해해보겠습니다.

영화 사이트 IMDB 영화 리뷰 데이터는 리뷰 텍스트에 리뷰가 긍정인 경우 1을, 부정인 경우 0으로 레이블링 한 데이터로 25,000개의 훈련 데이터와 테스트 데이터 25,000개로 구성된 데이터입니다. 케라스를 통해서 이 데이터셋을 바로 다운로드 할 수 있습니다. 이 데이터에 대한 상세 설명은 뒤의 RNN을 이용한 텍스트 분류 챕터에서 IMDB 리뷰 데이터를 다룰 때 설명합니다. 여기서는 데이터나 코드에 대한 설명보다는 단어 벡터의 평균으로 얻을 수 있는 성능에 주목해주세요.

---
## 1. 데이터 로드와 전처리
---

In [ ]:
import tensorflow as tf
tf.__version__

'2.6.0'

### 1. 데이터 로드와 전처리

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.datasets import imdb

케라스에서는 imdb.data_load()를 통해서 IMDB 리뷰 데이터를 다운로드 할 수 있는데, 데이터를 로드할 때 파라미터로 num_words를 사용하면 이 데이터에서 등장 빈도 순위로 몇 번째에 해당하는 단어까지를 사용할 것인지를 의미합니다. 만약 vocab_size를 20,000으로 지정할 경우 등장 빈도 순위가 20,000등이 넘는 단어들은 데이터를 로드할 때 전부 제거 후 로드합니다.

In [ ]:
vocab_size = 20000

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)
print('훈련용 리뷰 개수 :',len(X_train))
print('테스트용 리뷰 개수 :',len(X_test))

훈련용 리뷰 개수 : 25000
테스트용 리뷰 개수 : 25000


이 데이터는 이미 정수 인코딩까지의 전처리가 진행되어져 있습니다. 그래서 단어 집합을 만들고, 각 단어를 정수로 인코딩하는 과정을 할 필요가 없습니다. 첫번째 리뷰와 첫번째 리뷰의 레이블을 출력해봅시다.

In [ ]:
print('훈련 데이터의 첫번째 샘플 :', X_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [ ]:
print('훈련 데이터의 첫번째 샘플의 레이블 :'y_train[0])

1


정수 1이 출력되는데 이는 첫번째 리뷰가 긍정 리뷰임을 의미합니다. 각 리뷰의 평균 길이를 계산해봅시다.

In [ ]:
print('훈련용 리뷰의 평규 길이: {}'.format(np.mean(list(map(len, X_train)), dtype=int)))
print('테스트용 리뷰의 평균 길이: {}'.format(np.mean(list(map(len, X_test)), dtype=int)))

훈련용 리뷰의 평규 길이: 238
테스트용 리뷰의 평균 길이: 230


훈련용 리뷰와 테스트용 리뷰의 평균 길이가 각각 238, 230입니다. 평균보다는 큰 수치인 400으로 패딩합니다.

In [ ]:
max_len = 400

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
print('x_train의 크기(shape) :', X_train.shape)
print('x_test의 크기(shape) :', X_test.shape)

x_train의 크기(shape) : (25000, 400)
x_test의 크기(shape) : (25000, 400)


---
### 2. 모델 설계하기
---
모델의 입력으로 사용하기 위한 모든 전처리를 마쳤습니다. 임베딩 벡터를 평균으로 사용하는 모델을 설계해봅시다. GlobalAveragePooling1D()은 입력으로 들어오는 모든 벡터들의 평균을 구하는 역할을 합니다. **Embedding() 다음에 GlobalAveragePooling1D()** 을 추가하면 **해당 문장의 모든 단어 벡터들의 평균 벡터를 구합니다.**

이진 분류를 수행해야 하므로 그 후에는 시그모이드 함수를 활성화 함수로 사용하는 뉴런 1개를 배치합니다. 훈련 데이터의 20%를 검증 데이터로 사용하고 총 10 에포크 학습합니다.

In [ ]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

embedding_dim = 64

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))

# 모든 단어 벡터의 평균을 구한다.
model.add(GlobalAveragePooling1D())
model.add(Dense(1, activation='sigmoid'))

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('embedding_average_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
%time model.fit(X_train, y_train, batch_size=32, epochs=10, callbacks=[es, mc], validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 4s 3ms/step - loss: 0.6291 - acc: 0.7186 - val_loss: 0.5225 - val_acc: 0.8120

Epoch 00001: val_acc improved from -inf to 0.81200, saving model to embedding_average_model.h5
Epoch 2/10
625/625 [==============================] - 2s 3ms/step - loss: 0.4304 - acc: 0.8557 - val_loss: 0.3788 - val_acc: 0.8690

Epoch 00002: val_acc improved from 0.81200 to 0.86900, saving model to embedding_average_model.h5
Epoch 3/10
625/625 [==============================] - 2s 3ms/step - loss: 0.3190 - acc: 0.8911 - val_loss: 0.3233 - val_acc: 0.8798

Epoch 00003: val_acc improved from 0.86900 to 0.87980, saving model to embedding_average_model.h5
Epoch 4/10
625/625 [==============================] - 2s 3ms/step - loss: 0.2621 - acc: 0.9085 - val_loss: 0.2982 - val_acc: 0.8852

Epoch 00004: val_acc improved from 0.87980 to 0.88520, saving model to embedding_average_model.h5
Epoch 5/10
625/625 [==============================] - 2s 3ms/step - loss: 0.224

학습이 끝난 후에 테스트 데이터에 대해서 평가합니다.

In [ ]:
loaded_model = load_model('embedding_average_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

782/782 [==============================] - 1s 1ms/step - loss: 0.2921 - acc: 0.8867

 테스트 정확도: 0.8867


별 다른 신경망을 추가하지 않고 단어 벡터의 평균만으로도 88.76%라는 준수한 정확도를 얻어냅니다.

마지막 편집일시 : 2022년 11월 14일 3:12 오후